# Entropy and Term Length

```yaml
Course:  DS 5001
module:  03 
topic:   Entropy and Term Length
author:  R.C. Alvarado
date:    12 Dec 2021
purpose: This notebook describes the relationship between term length and entropy.
```

# Set Up

## Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px

ModuleNotFoundError: No module named 'plotly_express'

In [ ]:
sns.set()

## Config

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_dir = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [ ]:
OHCO = ['book_id', 'chap_num', 'para_num', 'sent_num', 'token_num']

## Import Data

In [ ]:
K = pd.read_csv(f"{output_dir}/austen-combo-TOKENS.csv").set_index(OHCO)
V = pd.read_csv(f"{output_dir}/austen-combo-VOCAB.csv").set_index('term_str')

# Add some features for this analysis
V['s'] = 1 / V.p

In [ ]:
K.head()

In [ ]:
V.head()

# Vocab $V$

## Corpus frequency of terms

Some words are very high frequency, but the vast majority have very low frequencies. We will explore the significane of this difference in the next module. NB: By "frequency" we mean relative frequency, which we use to estimate $p$.

In [ ]:
V.p.sort_values(ascending=False).plot(style='.-', rot=45);

## Frequency of frequencies

The same data, but looked at in terms of the distribition of probabilities. This is so that it can be compared to the following graphs of $s$, $i$, and $h$.

In [ ]:
V.p.round(4).value_counts().sort_index().plot(style='.-');

# Chismus Structure of $p$, $s$, $i$, and $h$

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,5), sharey='row')
fig.tight_layout(pad=2)
V.p.value_counts().sort_index().plot(style='.-', ax=axes[0,0], title="A1: $p$") 
V.s.value_counts().sort_index().plot(style='.-', ax=axes[0,1], title="B1: $s$")
V.i.value_counts().sort_index().plot(style='.-', ax=axes[1,1], title="B2: $i$") 
V.h.value_counts().sort_index().plot(style='.-', ax=axes[1,0], title="A2: $h$");

Note the range of $i$: it is one the same order as the range of `n_chars`.

# Compare $L$ to $p$, $s$, $i$, and $h$

Now let's look at how these four features relate to term length $L$, the number of characters in a term (`n_chars`)

## Create a dataframe for $L$

In [ ]:
VG = V.groupby('n_chars')

# Distribution of L over the vocabulary (terms)
L = VG.n.count().to_frame('v_n')
L['v_p'] = L.v_n / L.v_n.sum()
L['v_s'] = 1 / L.v_p
L['v_i'] = np.log2(L.v_s)
L['v_h'] = L.v_p * L.v_i

# Distribution of L over the corpus (tokens)
L['k_n'] = VG.n.sum()
L['k_p'] = L.k_n / L.k_n.sum()
L['k_s'] = 1 / L.k_p
L['k_i'] = np.log2(L.k_s)
L['k_h'] = L.k_p * L.k_i

# Aggregate probability features over tokens
for func in ['sum','mean']:
    for x in 'psih':
        L[f"k{func}_{x}"] = VG[x].agg(func)    
        
# L.index.name = 'n_chars'        
L.columns = pd.Index([tuple(col.split('_')) for col in L.columns])
L.T.index.names = ['pop','stat']
Lnorm = (L - L.mean()) / L.std()

## $L$ distributions

Compare corpus and vocab frequencies. Why are the distributions different?

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey='row', figsize=(10,5))
L.v.p.plot.bar( ax=axes[0,0], title='L over Vocab')
L.k.p.plot.bar( ax=axes[0,1], title='L over Corpus')
Lnorm.v.p.plot.bar( ax=axes[1,0], rot=0)
Lnorm.k.p.plot.bar( ax=axes[1,1], rot=0);

In [ ]:
L.style.background_gradient()

In [ ]:
Lnorm.v.plot(figsize=(20,5))

In [ ]:
Lnorm.ksum.plot(figsize=(20,5));

**NOTE:** Notice how easy it is to combare features in the shaded table vs the chart.

## Plot All

In [ ]:
fig, axes = plt.subplots(ncols=4, nrows=6, figsize=(20,20), sharex=True)
fig.tight_layout(pad=2.5)
for i, x in enumerate(list("psih")):
    
#     ax = axes[0, i]
    
#     # Counts 
#     V[x].value_counts().plot(style='*-', ax=ax)
#     ax.set_title(f"${x}$")    

    ax = axes[0, i]
    
    # Means and points
    V.plot('n_chars', x, style='.', ax=ax, legend=False)
    L.kmean[x].plot(ax=ax);
    ax.set_title(f'corpus mean {x}')

    ax = axes[1, i]
    
    # Variances 
    sns.boxplot(data=V.reset_index(), x='n_chars', y=x, ax=ax)
    ax.set_title(f'corpus variance {x}')

    ax = axes[2, i]

    # Ksum
    L.ksum[x].plot(ax=ax)
    ax.set_title(f'corpus sum {x}')
    
    ax = axes[3, i]
    
    # Kmean
    L.kmean[x].plot(ax=ax)
    ax.set_title(f"corpus mean {x}")

    ax = axes[4, i]
    
    # K
    L.k[x].plot(ax=ax)
    ax.set_title(f"corpus {x}")

    ax = axes[5, i]
    
    # V
    L.v[x].plot(ax=ax)
    ax.set_title(f"vocab {x}")


In [ ]:
def plot_chiasmus(df, kind='bar'):
    fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,6), sharex=True)
    fig.tight_layout()
    df.p.plot(kind=kind, ax=axes[0,0], title='P')
    df.s.plot(kind=kind, ax=axes[0,1], title='S')
    df.i.plot(kind=kind, ax=axes[1,1], title='I', rot=0)
    df.h.plot(kind=kind, ax=axes[1,0], title='H', rot=0)

In [ ]:
# plot_chiasmus(Lnorm.v, 'bar')

In [ ]:
# plot_chiasmus(Lnorm.k, 'bar')

In [ ]:
# plot_chiasmus(Lnorm.ksum, 'bar')

In [ ]:
# plot_chiasmus(Lnorm.kmean, 'bar')

# Conclusions

* All high frequency, low information terms are short.
* Average term length increase with information.
* Information and term length in fact have the same scale! They are weakly correlated; $0.3$ 
  * Characters as units of information.
* $s$ or $i$ can be used to cull significant terms.
* The bulk of surprise is in the seven letter words. 
  * Does this mean that the vocabulary frequency of character length is short cut to computing the entropy of character length types?

# Explore Term Length Vector Space 

Can we learn anything about the structure of our corpus from distribution of term lengths? If information is related to length, there must be something to observe.

In [ ]:
local_lib = config['DEFAULT']['local_lib']
import sys; sys.path.append(local_lib)
from hac import HAC

In [ ]:
K['n_chars'] = K.term_str.str.len()

In [ ]:
T1 = K.groupby(OHCO[:2]+['n_chars']).n_chars.count().unstack(fill_value=0)

T2 = (T1.T / T1.T.sum()).T

T3 = T2.corr().stack().sort_values().to_frame('c')
T3.index.names = ['x','y']
T3 = T3[T3.apply(lambda x: x.name[0] > x.name[1], 1)]
T3 = T3.sort_values('c', ascending=False)

## Book 1 (SS)

In [ ]:
T1.loc[1, 5:].style.background_gradient(axis=1)

In [ ]:
sns.boxplot(data=T2.loc[1]);

In [ ]:
T2.loc[1, 1:].plot(figsize=(20,10), style='.-');

In [ ]:
HAC(T2.loc[1]).plot()

## Book 2 (P)

In [ ]:
T1.loc[2, 5:].style.background_gradient(axis=1)

In [ ]:
sns.boxplot(data=T2.loc[2]);

In [ ]:
# sns.relplot?

In [ ]:
sns.relplot(data=T2.loc[2], kind='line', aspect=2.5);

In [ ]:
HAC(T2.loc[2]).plot()